# E2E Deep Learning for AV Control Using Udacity's Car Simulator Environment

## Mina Rahmanian Shahri - 20137470

### Master of Queen's University

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## Install Dependencies


In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==2.0

In [ ]:
!pip uninstall keras
!pip install Keras==2.3.0

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
!python -c 'import keras; print(keras.__version__)'

In [ ]:
!pip3 install imgaug

## Importing Python libraries


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle
import matplotlib.image as mpimg
import os
import gc
import ntpath
import json
from imgaug import augmenters as iaa
import skimage.transform as sktransform


# keras
import keras
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Convolution2D, MaxPooling2D,Dense,Flatten,Dropout
from keras import backend as K


Unzip Dataset

In [ ]:
!unzip '/content/gdrive/My Drive/car/data.zip' -d '/content/gdrive/My Drive/car'

## Load data

In [ ]:
# Columns: ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']


dataload = '/content/gdrive/My Drive/car/data'

data_dir = pd.read_csv(os.path.join(dataload, 'driving_log.csv'))
pd.set_option('display.max_colwidth', -1)


data_dir.head(5)

In [ ]:

def path_leaf(path):
  head, tail = ntpath.split(path)
  return tail


data_dir['center'] = data_dir['center'].apply(path_leaf)
data_dir['left'] = data_dir['left'].apply(path_leaf)
data_dir['right'] = data_dir['right'].apply(path_leaf)


data_dir.head(200)

In [ ]:
data_dir.describe()

## Signal with `savgol_filter`

In [ ]:
# Smooth data signal with `savgol_filter`
import scipy
from scipy import signal


# data_dir["steering"] = signal.savgol_filter(data_dir["steering"].values.tolist(), 43, 17)
data_dir["steering"] = signal.savgol_filter(data_dir["steering"].values.tolist(), 51, 11)
data_dir["speed"] = signal.savgol_filter(data_dir["speed"].values.tolist(), 25, 15)


In [ ]:
data_dir.head(200)

In [ ]:
data_dir.describe()

## Dataset distribution

In [ ]:
# Data distribution
new=data_dir['steering']
steer=list(new)


plt.figure(figsize=(14,8))
x = [range(len(steer))]
x = np.squeeze(np.asarray(x))
y = np.asarray(steer)


# plot
plt.xlim(0,8000)
plt.title('Data Distribution', fontsize=22)
plt.xlabel('Frames', fontsize=16)
plt.ylabel('Steering Angle', fontsize=16)
plt.plot(x,y, 'g', linewidth=0.4)
plt.show()


## Dataset Balance

In [ ]:
# Visualize data
num_bins = 25
samples_per_bin = 180  #400  #200

hist, bins = np.histogram(data_dir['steering'], num_bins)
center = (bins[:-1]+ bins[1:]) * 0.5

# Plot
fig= plt.figure(figsize=(10,7))
plt.bar(center, hist, width=0.05)
plt.plot((np.min(data_dir['steering']), np.max(data_dir['steering'])), (samples_per_bin, samples_per_bin),'r')
plt.title('Before balance control', fontsize=22)
plt.xlabel('steering angle', fontsize=15)
plt.ylabel('counts', fontsize=15)

In [ ]:
data = data_dir
print('total data: ',len(data))

# Make list of indices to remove
remove_list=[]

for j in range(num_bins):
    list_=[]
    for i in range(len(data['steering'])):
        if data['steering'][i] > bins[j] and data['steering'][i] <= bins[j+1]:
            list_.append(i)
    list_=shuffle(list_)
    list_=list_[samples_per_bin:]
    remove_list.extend(list_)


# Remove from extras from list   
print('removed data: ',len(remove_list))
data.drop(data.index[remove_list],inplace=True)
print('remainig data: ',len(data))


# Plot
hist, _ = np.histogram(data['steering'],(num_bins))

fig= plt.figure(figsize=(10,7))
plt.bar(center,hist,width=0.05)
plt.plot((np.min(data['steering']),np.max(data['steering'])),(samples_per_bin,samples_per_bin),'r')

# Display the balance of each columns of dataset

In [ ]:

balanced = pd.DataFrame()   # Balanced dataset
bins = 1000                 # N of bins
bin_n = 200                 # N of examples to include in each bin (at most)

start = 0
for end in np.linspace(0, 1, num=bins):  
    data_range = data[(np.absolute(data.steering) >= start) & (np.absolute(data.steering) < end)]
    range_n = min(bin_n, data_range.shape[0])
    balanced = pd.concat([balanced, data_range.sample(range_n)])
    start = end
    
balanced.to_csv('/content/gdrive/My Drive/car/data/driving_log.csv', index=False)



balanced.plot.hist(grid=True, bins=200, rwidth=3.5)

fig = plt.figure(figsize = (12,12))
ax = fig.gca()
balanced.hist(ax=ax)
plt.show()



# Create Dataset 

In [ ]:
print(data.iloc[1])

# Get images and measurements from left/right camera
# Get images and steering data into arrays
def load_img_steering(datadir, df):
  image_path = []
  steering = []

  for i in range(len(data)):
    indexed_data = data.iloc[i]
    center, left, right = indexed_data[0], indexed_data[1], indexed_data[2]
    image_path.append(os.path.join(datadir, center.strip()))
    steering.append(float(indexed_data[3]))

    # left image append
    image_path.append(os.path.join(datadir,left.strip()))
    steering.append(float(indexed_data[3])+ 0.25) # 0.15
    
    # right image append
    image_path.append(os.path.join(datadir,right.strip()))
    steering.append(float(indexed_data[3])- 0.25) # 0.15


  image_paths = np.asarray(image_path)
  steerings = np.asarray(steering)
  return image_paths, steerings
 
 
image_paths, steerings = load_img_steering(dataload + '/IMG', data)


## Randomly display the Images 

In [ ]:

im1 = image_paths[random.randint(0, 1000)]
im2 = image_paths[random.randint(0, 1000)]
im3 = image_paths[random.randint(0, 1000)]

imshow1= mpimg.imread(im1)
imshow2= mpimg.imread(im2)
imshow3= mpimg.imread(im3)

fig, axs = plt.subplots(1,3, figsize=(20, 10))

axs[0].imshow(imshow1)
axs[0].set_title('Path 1 ')
axs[1].imshow(imshow2)
axs[1].set_title('Path 2 ')
axs[2].imshow(imshow3)
axs[2].set_title('Path 3 ')


h1, im1= ntpath.split(im1)
h2, im2= ntpath.split(im2)
h3, im3= ntpath.split(im3)


print('path 1 = {}'.format (im1)) 
print('path 2 = {}'.format (im2))
print('path 3 = {} \n\n'.format (im3))

 Data Selection:
## Spliting Training & Validation

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(image_paths, steerings, test_size=0.25, random_state=6)
print('Training Samples: {}\nValid Samples: {}'.format(len(X_train), len(X_valid)))

In [ ]:
fig,axes=plt.subplots(1,2,figsize=(18,7))

axes[0].hist(Y_train,bins=num_bins,width=0.05,color='royalblue')
axes[0].set_title('Trainig Set')

axes[1].hist(Y_valid,bins=num_bins,width=0.05,color='tomato')
axes[1].set_title('Validation Set')


#Data Augmentation



Zoom Augmentation

In [ ]:
# Get any image
def zoom(image):
  zoom = iaa.Affine(scale=(1, 1.3))
  image = zoom.augment_image(image)
  
  return image


# Zoom random images
# Compare original and zoom image
image = image_paths[random.randint(0, 500)]
original_image = mpimg.imread(image)
zoomed_image = zoom(original_image)
 
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
 
axs[0].imshow(original_image)
axs[0].set_title('Original Image')
 
axs[1].imshow(zoomed_image)
axs[1].set_title('Zoomed Image')  

Translational Augmentation

In [ ]:
# Get any image
def pan(image):
  pan = iaa.Affine(translate_percent= {"x" : (-0.1, 0.1), "y": (-0.1, 0.1)})
  image = pan.augment_image(image)
  return image

# Translating random images
# Compare original and panned/Translational image
image = image_paths[random.randint(0, 500)]
original_image = mpimg.imread(image)
panned_image = pan(original_image)
 
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
 
axs[0].imshow(original_image)
axs[0].set_title('Original Image')
 
axs[1].imshow(panned_image)
axs[1].set_title('Panned Image')  

Brightness Augmentation

In [ ]:
# Get any image
def img_random_brightness(image):
    brightness = iaa.Multiply((0.2, 1.2))
    image = brightness.augment_image(image)
    return image

# Altering the brightness of random images
# Compare original and brightness image
image = image_paths[random.randint(0, 500)]
original_image = mpimg.imread(image)
brightness_altered_image = img_random_brightness(original_image)
 
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
 
axs[0].imshow(original_image)
axs[0].set_title('Original Image')
 
axs[1].imshow(brightness_altered_image)
axs[1].set_title('Brightness altered image ')    

Flipping Augmentation

In [ ]:
# Get any image
def img_random_flip(image, steering_angle):
    image = cv2.flip(image,1)
    steering_angle = -steering_angle
    
    return image, steering_angle


# Randomly flipping the input image horizontaly, with steering angle adjustment.
# Compare original and Flipping image
random_index = random.randint(0, 500)
image = image_paths[random_index]
steering_angle = steerings[random_index]
 
 
original_image = mpimg.imread(image)
flipped_image, flipped_steering_angle = img_random_flip(original_image, steering_angle)
 
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
 
axs[0].imshow(original_image)
axs[0].set_title('Original Image - ' + 'Steering Angle: ' + str(steering_angle))
 
axs[1].imshow(flipped_image)
axs[1].set_title('Flipped Image - ' + 'Steering Angle: ' + str(flipped_steering_angle))    

In [ ]:
# Generate an augumented image and adjust the associated steering angle.
def random_augment(image, steering_angle):

    image = mpimg.imread(image)
    if np.random.rand() < 0.5:
      image = pan(image)
    if np.random.rand() < 0.5:
      image = zoom(image)
    if np.random.rand() < 0.5:
      image = img_random_brightness(image)
    if np.random.rand() < 0.5:
      image, steering_angle = img_random_flip(image, steering_angle)
    
    
    return image, steering_angle

In [ ]:
ncol = 2
nrow = 10
 
fig, axs = plt.subplots(nrow, ncol, figsize=(15, 70))
fig.tight_layout()

# Compare some original and augumented images 
for i in range(10):
  randnum = random.randint(0, len(image_paths) - 1)
  random_image = image_paths[randnum]
  random_steering = steerings[randnum]
    
  original_image = mpimg.imread(random_image)
  augmented_image, steering = random_augment(random_image, random_steering)
    
  axs[i][0].imshow(original_image)
  axs[i][0].set_title("Original Image")
  
  axs[i][1].imshow(augmented_image)
  axs[i][1].set_title("Augmented Image")

# Data Preprocessing

In [ ]:
def img_preprocess(img):

    # Cropping the image
    img = img[60:135,:,:]
    # Converting the image to the YUV color space.
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img,  (3, 3), 0)
    # Resizing the image to (66 * 200), the image size that the model expects.
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img

In [ ]:
image = image_paths[100]
original_image = mpimg.imread(image)
preprocessed_image = img_preprocess(original_image)
 
# Compare original and preprocessing image
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
axs[0].imshow(original_image)
axs[0].set_title('Original Image')
axs[1].imshow(preprocessed_image)
axs[1].set_title('Preprocessed Image')

### Generate and Compare an Augumentation and Preprocessing image in Training & Validation separately.

In [ ]:
def batch_generator(image_paths, steering_ang, batch_size, istraining):

# batch_size =100

  while True:
    batch_img = []
    batch_steering = []
    
    for i in range(batch_size):
      random_index = random.randint(0, len(image_paths) - 1)
      
      if istraining:
        im, steering = random_augment(image_paths[random_index], steering_ang[random_index])
     
      else:
        im = mpimg.imread(image_paths[random_index])
        steering = steering_ang[random_index]
      
      im = img_preprocess(im)
      batch_img.append(im)
      batch_steering.append(steering)
    yield (np.asarray(batch_img), np.asarray(batch_steering))

In [ ]:
x_train_gen, y_train_gen = next(batch_generator(X_train, Y_train, 1, 1))
x_valid_gen, y_valid_gen = next(batch_generator(X_valid, Y_valid, 1, 0))
 
fig, axs = plt.subplots(1, 2, figsize=(15, 10))
fig.tight_layout()
 
axs[0].imshow(x_train_gen[0])
axs[0].set_title('Training Image')
 
axs[1].imshow(x_valid_gen[0])
axs[1].set_title('Validation Image')

# Define models

In [ ]:
'''

## Define Swish activation function

#t = -1.0
#def ftswish(x):
#  return K.maximum(t, K.relu(x)*K.sigmoid(x) + t)

##########################################################


def nvidia_model():
  model = Sequential()
  model.add(Convolution2D(24, 5, 5, subsample=(2, 2), input_shape=(66, 200, 3), activation='elu'))
  model.add(Convolution2D(36, 5, 5, subsample=(2, 2), activation='elu'))
  model.add(Convolution2D(48, 5, 5, subsample=(2, 2), activation='elu'))
  model.add(Convolution2D(64, 3, 3, activation='elu'))
  
  model.add(Convolution2D(64, 3, 3, activation='elu'))
#   model.add(Dropout(0.5))
  
  
  model.add(Flatten())
  
  model.add(Dense(100, activation = 'elu'))
#   model.add(Dropout(0.5))
  
  model.add(Dense(50, activation = 'elu'))
#   model.add(Dropout(0.5))
  
  model.add(Dense(10, activation = 'elu'))
#   model.add(Dropout(0.5))
 
  model.add(Dense(1))
  
  # Define optimizer and criterion
  optimizer = Adam(lr=1e-3)
  model.compile(loss='mse', optimizer=optimizer, metrics=['mean_squared_error'])
  return model


  '''

In [ ]:
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import csv
import cv2
import matplotlib.pyplot as plt
import random
import matplotlib.image as mpimg
from keras.layers import Cropping2D
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from tensorflow.keras import regularizers
from keras.regularizers import l2
from keras.layers import Cropping2D
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from keras.constraints import max_norm
from keras import backend as K
from keras.backend import sigmoid 
from keras.utils.generic_utils import get_custom_objects 
from keras.layers import Activation 
from keras import backend as K
from keras.applications import ResNet50
from keras.applications import VGG16



def nvidia_model2():
    model = Sequential()
    model.add(Lambda(lambda x: x/255.0 -0.5, input_shape=(66, 200, 3))) # normalization

    model.add(BatchNormalization())
    model.add(Conv2D(24, 5, 5, activation= 'relu', subsample=(2, 2), kernel_regularizer=regularizers.l2(0.001), kernel_constraint=max_norm(3), bias_constraint=max_norm(3)))

    model.add(BatchNormalization())
    model.add(Conv2D(36, 5, 5, activation='relu', subsample=(2, 2), kernel_regularizer=regularizers.l2(0.001), kernel_constraint=max_norm(3), bias_constraint=max_norm(3)))

    model.add(BatchNormalization())
    model.add(Conv2D(48, 5, 5, activation='relu', subsample=(2, 2), kernel_regularizer=regularizers.l2(0.001), kernel_constraint=max_norm(3), bias_constraint=max_norm(3)))

    model.add(BatchNormalization())
    model.add(Conv2D(64, 3, 3, activation='relu', kernel_regularizer=regularizers.l2(0.001), kernel_constraint=max_norm(3), bias_constraint=max_norm(3))) 

    model.add(BatchNormalization())
    model.add(Conv2D(64, 3, 3, activation='relu', kernel_regularizer=regularizers.l2(0.001), kernel_constraint=max_norm(3), bias_constraint=max_norm(3))) 


    model.add(Flatten())


    model.add(Dropout(0.4))
    model.add(Dense(100, activation='relu'))

    model.add(Dropout(0.4))
    model.add(Dense(50, activation='relu'))

    model.add(Dropout(0.4))
    model.add(Dense(10, activation='relu'))


    model.add(Dense(1))

 
    # Define optimizer and criterion
    optimizer = Adam(lr=1e-3)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mean_squared_error'])
    return model


 I can not try below cell because of computational limitation

In [ ]:
'''

# vgg = VGG16(weights='imagenet', include_top=False, input_shape=(66, 200, 3))

resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(66, 200, 3))

for layer in resnet.layers[:-4]:
    layer.trainable = False
 
for layer in resnet.layers:
    print(layer, layer.trainable)


def nvidia_model3():
    model = Sequential()
    #model.add(Lambda(lambda x: x / 127.5 - 1., input_shape=(66, 200, 3))) # normalization

    model.add(resnet)
    # model.add(vgg)
    model.add(Dropout(0.4))

    model.add(Flatten())

    model.add(Dropout(0.4))
    model.add(Dense(100, activation='relu'))

    model.add(Dropout(0.4))
    model.add(Dense(50, activation='relu'))

    model.add(Dropout(0.4))
    model.add(Dense(10, activation='relu'))


    model.add(Dense(1))


    # Define optimizer and criterion
    optimizer = Adam(lr=1e-3)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mean_squared_error'])
    return model

'''

# Architectures of Model/Network

In [ ]:
model=nvidia_model2()
print(model.summary())

# Train Model

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

filepath="E2E_cnn.hdf5"


checkpoint1 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#checkpoint2 = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5)
callbacks_list = [checkpoint1]

# Set callback functions to early stop training and save the best model so far
#callbacks_list = [checkpoint1, checkpoint2]


# Train Network
history = model.fit_generator(batch_generator(X_train, Y_train, 100, 1),
                                  steps_per_epoch=400, 
                                  epochs=50,
                                  validation_data=batch_generator(X_valid, Y_valid, 100, 0),
                                  validation_steps=300,
                                  verbose=1,
                                  shuffle = 1,
                                  callbacks = callbacks_list)


## Plot Training Loss vs Validation Loss

In [ ]:

fig= plt.figure(figsize=(12,7))

plt.title("Training Loss vs Validation Loss")
plt.xlabel("Iterations/Epoch")
plt.plot(history.history['loss'], 'b')
plt.plot(history.history['val_loss'], 'g-.')
plt.legend(['Training Loss','Validation Loss'])
plt.grid()
plt.show()

## Plot Training MSE vs Validation MSE

In [ ]:

fig= plt.figure(figsize=(12,7))
plt.title("Training MSE vs Validation MSE")
plt.xlabel("Iterations/Epoch")
plt.plot(history.history['mean_squared_error'], 'b')
plt.plot(history.history['val_mean_squared_error'], 'g-.')
plt.legend(['Training Loss','Validation Loss'])
plt.grid()
plt.show()

## Save Model - Download Model

In [ ]:
model.save('modelt.h5')

In [ ]:
from google.colab import files
files.download('modelt.h5')

In [ ]:
# model.save('model6.json')

In [ ]:
# from google.colab import files
# files.download('model6.json')